In [1]:
from init import *

In [2]:
def BuildModel_Simple():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [3]:
m = BuildModel_Simple()
m.evaluations.append(sdk.EvaluationPoint(2))
m.json()

'{"version": "Model:1", "TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "BrownianMotion", "refs": [], "args": [0.2, 2], "start": [0.1]}], "evaluations": [{"time": 2}]}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'histograms': [],
 'mean': [0.4896470606327057],
 'names': ['BrownianMotion'],
 'npaths': [10000],
 'skewness': [-0.019155969843268394],
 'stddev': [2.7979307174682617],
 'time_points': [2.0],
 'time_steps': [9]}

In [5]:
import time
m = BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 1000
m.RunTimeoutSeconds = 16.7
for t in [1,2,5,10]:
    m.evaluations.append(sdk.EvaluationPoint(t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'histograms': [],
 'mean': [0.3003929853439331,
  0.4978606104850769,
  1.1002122163772583,
  2.0956273078918457],
 'names': ['BrownianMotion'],
 'npaths': [1000000, 1000000, 1000000, 1000000],
 'skewness': [-0.0009215424070134759,
  0.00028438729350455105,
  -0.0024296639021486044,
  -0.003244265215471387],
 'stddev': [1.999655842781067,
  2.8266067504882812,
  4.470344066619873,
  6.325104236602783],
 'time_points': [1.0, 2.0, 5.0, 10.0],
 'time_steps': [99, 199, 499, 999]}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

2.7 nanoseconds to make a single time step


# Two processes, Three evaluations

In [7]:
def TwoProcessesModel ():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [8]:
model = sdk.Model()
model.TimeStart = 0
model.TimeSteps = 10
model.NumPaths = 10000
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.TimeSteps = 10000
for t in [1,2,5]:
    model.evaluations.append(sdk.EvaluationPoint(time=t))
r = requests.post(f'{server}/model',model.json())
r.json()

{'histograms': [],
 'mean': [0.3405849039554596,
  0.2630581855773926,
  0.5352856516838074,
  0.4589063823223114,
  1.2040940523147583,
  1.013039469718933],
 'names': ['BrownianMotion', 'BrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'skewness': [-0.016896016895771027,
  -0.009573712013661861,
  -0.027073148638010025,
  0.013739141635596752,
  0.009943977929651737,
  0.017942816019058228],
 'stddev': [1.9741570949554443,
  2.011237621307373,
  2.7974250316619873,
  2.840625762939453,
  4.454368591308594,
  4.43344259262085],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [1999, 3999, 9999]}

In [9]:
results = sdk.run (model, server)
results.df()

,name,title,state,point,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,0,0,1.0,1999,10000,0.340585,0.019742,1.974157,-0.016896
1,BrownianMotion,,1,0,1.0,1999,10000,0.263058,0.020112,2.011238,-0.009574
2,BrownianMotion,,0,1,2.0,3999,10000,0.535286,0.027974,2.797425,-0.027073
3,BrownianMotion,,1,1,2.0,3999,10000,0.458906,0.028406,2.840626,0.013739
4,BrownianMotion,,0,2,5.0,9999,10000,1.204094,0.044544,4.454369,0.009944
5,BrownianMotion,,1,2,5.0,9999,10000,1.013039,0.044334,4.433443,0.017943
